In [1]:
# ----------------------------------------------------------------------------
#
# TITLE - test_matrix_multiplication.ipynb
# AUTHOR - James Lane
# PROJECT - AST 1501
#
# ----------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Make sure the prior on our halo angle works like expected by testing out matrix multiplication in python 
with Hogg+ 2010 data.
'''

__author__ = "James Lane"

In [2]:
import numpy as np
from scipy import linalg
import scipy as sc
from astropy.table import Table
from matplotlib import pyplot as plt
import pdb

# Prepare

In [3]:
# Read Hogg+ 2010 data.
data = np.genfromtxt('hogg_2010.dat', comments='#', delimiter=' & ')
xs = data[:,1]
ys = data[:,2]
ysigs = data[:,3]
ndata = len(xs)
print(ndata)

20


In [4]:
# Make arrays
Y = np.zeros((ndata,1)) # A column vector
A = np.zeros((ndata,2)) # An Nx2 matrix
C = np.zeros((ndata,ndata)) # An NxN matrix

# Fill arrays
for i in range( ndata ):
    Y[i] = ys[i]
    A[i,0] = 1
    A[i,1] = xs[i]
    C[i,i] = ysigs[i]**2
###i

In [5]:
# Make sure everything has the right shape
print(Y.shape)
print(A.shape)
print(C.shape)

(20, 1)
(20, 2)
(20, 20)


In [6]:
# Make some convenience matrices
C_inv = np.linalg.inv(C)

# Calculizzle

First calculate $X$, the column vector containing $b,m$ which are the best-fitting parameters for the linear profile $y=mx+b$

The equation is:

$
X = [A^{T}C^{-1}A]^{-1}[A^{T}C^{-1}Y]
$

First compute $A^{T}C^{-1}Y$

In [7]:
rt = np.matmul( A.T, np.matmul( C_inv, Y ) )

Now compute $[A^{T}C^{-1}A]^{-1}$

In [8]:
lt = np.linalg.inv( np.matmul( A.T, np.matmul( C_inv, A ) ) )

Calculate the final product: $[b,m]$

In [9]:
X = np.matmul( lt, rt )
print(X)
print( np.sqrt( np.diag(lt) ) )

[[213.27349198]
 [  1.07674752]]
[14.39403311  0.07740678]


# Test linear model
The solution to $p(\phi_{b}|Data)$ seems strange, so we'll have to check it with the Hogg data.

We have that:
$
\chi^{2} = Y^{T}C^{-1}Y - Y^{T}C^{-1}AX -X^{T}A^{T}C^{-1}Y + X^{T}A^{T}C^{-1}AX
$

and we would like to match it to the form:
$
\eta^{2} = W^{T}V^{-1}W - W^{T}V^{-1}X - X^{T}V^{-1}W + X^{T}V^{-1}X
$

With the apparent solution being: $ V^{-1} = A^{T}C^{-1}A $, $W^{T} = Y^{T}C^{-1}A[A^{T}C^{-1}A]^{-1}$, $W = [A^{T}C^{-1}A]^{-1}A^{T}C^{-1}Y$.

There is also a hanging term $U = Y^{T}C^{-1}[Y-AW]$

Let's calculate these terms and see if these equations match.

In [13]:
# First calculate chi square term
ct1 = np.linalg.multi_dot( [Y.T,C_inv,Y] )
ct2 = np.linalg.multi_dot( [Y.T,C_inv,A,X] )
ct3 = np.linalg.multi_dot( [X.T,A.T,C_inv,Y] )
ct4 = np.linalg.multi_dot( [X.T,A.T,C_inv,A,X] )

chisq = ct1 - ct2 - ct3 + ct4

In [14]:
# calculate the new variables
V_inv = np.linalg.multi_dot( [ A.T, C_inv, A ] )
W = np.matmul( np.linalg.inv( np.linalg.multi_dot( [A.T,C_inv,A] ) ), np.linalg.multi_dot( [A.T,C_inv,Y] ) )
W_T = np.matmul( np.linalg.multi_dot( [Y.T,C_inv,A] ), np.linalg.inv( np.linalg.multi_dot( [A.T,C_inv,A] ) ) )

# now calculate the eta square term
et1 = np.linalg.multi_dot( [W_T,V_inv,W] )
et2 = np.linalg.multi_dot( [W_T,V_inv,X] )
et3 = np.linalg.multi_dot( [X.T,V_inv,W] )
et4 = np.linalg.multi_dot( [X.T,V_inv,X] )
u = np.linalg.multi_dot( [Y.T,C_inv,Y-np.matmul(A,W)] )

etasq = et1 - et2 - et3 + et4 + u

In [15]:
print(chisq)
print(etasq)

[[289.96372278]]
[[289.96372278]]


Is the calculation of $W$ even correct?